In [1]:
from importlib import reload
import pandas as pd
import time
import TextPreprocessing as wpp
import CountryExtractor as ce
import SentimentAnalyser as sa
import TopicCategorizer as tc

Reading raw data

In [19]:
data = pd.read_csv('../data/new_data_vdss.tsv.xz', sep='\t', compression='xz')

In [20]:
# Datum bereinigen & redundante Spalten löschen

data['datum'] = data['pubtime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
data['datum'] = pd.to_datetime(data['datum'])
del data['pubtime']
del data['medium_code']
del data['regional']
del data['doctype']
del data['language']
del data['char_count']
del data['dateline']
del data['subhead']
del data['content_id']

Text preprocessing for content

In [ ]:
preprocessor = wpp.headlinePreprocessing()

start_time = time.time()
data['content_processed'] = data['content'].apply(lambda x: preprocessor.preprocess_text(x) if isinstance(x, str) else [])
end_time = time.time()

elapsed_time = end_time - start_time

del data['content']

print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

Country analysis with running time estimation

In [23]:
extractor = ce.CountryExtractor()
%timeit extractor.get_country(data['content'][0])

68.1 µs ± 9.06 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


68.1 µs ± 9.06 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

In [ ]:
# extraction of countries
extractor = ce.CountryExtractor()
data['countries'] = data['content_processed'].apply(lambda x: extractor.get_country(x) if isinstance(x[0], str) else [])

16.2 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

Sentiment analysis with running time estimation

In [31]:
reload(sa)
sentiment_analyser = sa.sentimentAnalyser()
%timeit sentiment_analyser.get_topic_sentiments_polarity(data['content_processed'][0])

65.9 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


12.8 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

In [33]:
# Sentiment analysis

sentiment_analyser = sa.sentimentAnalyser()

start_time = time.time()
data['Sentiment'] = data['content_processed'].apply(lambda x: sentiment_analyser.get_topic_sentiments_polarity(x))
end_time = time.time()

elapsed_time = end_time - start_time

print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

data.to_csv('../data/processed_data.tsv.xz', sep='\t', index=False, compression='xz')

Elapsed Time: 7932.10 seconds


Elapsed Time: 7932.10 seconds

Subjectivity analysis with running time estimation

In [73]:
reload(sa)
sentiment_analyser = sa.sentimentAnalyser()
%timeit sentiment_analyser.get_topic_subjectivity(processed_data['content_processed'][0])

16.4 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


16.4 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

In [75]:
sentiment_analyser = sa.sentimentAnalyser()

start_time = time.time()

data['objectivity'] = data['content_processed'].apply(lambda x: sentiment_analyser.get_topic_subjectivity(x))

end_time = time.time()

elapsed_time = end_time - start_time

print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

data.to_csv('../data/processed_data2.tsv.xz', sep='\t', index=False, compression='xz')

Elapsed Time: 9175.09 seconds


Elapsed Time: 9175.09 seconds

Text preprocessing 'rubric' and running time estimation

In [144]:
text_processing = wpp.TextPreprocessing()
%timeit data['rubric'][:1].apply(lambda x: text_processing.preprocess_text(x.lower()))

330 µs ± 16.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


330 µs ± 16.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

In [150]:
data['topic'] = data['rubric'].apply(lambda x: text_processing.preprocess_text(x.lower()) if isinstance(x, str) else [])

Import of processed data

In [46]:
import TopicCategorizer as tc
reload(tc)
%timeit data['content_processed'][:1].apply(lambda x: categorizer.categorize(x))

14 ms ± 899 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


14 ms ± 899 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

In [47]:
categorizer = tc.TopicCategorizer()

start_time = time.time()
data['kategorie'] = data['content_processed'].apply(lambda x: categorizer.categorize(x))
end_time = time.time()

elapsed_time = end_time - start_time

print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

data.to_csv('../data/processed_data3.tsv.xz', sep='\t', index=False, compression='xz')

Elapsed Time: 7089.20 seconds


Elapsed Time: 7089.20 seconds

In [38]:
processed_data = pd.read_csv('../data/processed_data3.tsv.xz', sep='\t', compression='xz')

In [40]:
# eval converts string to list
processed_data['content_processed'] = processed_data['content_processed'].apply(eval)

,id,medium_name,rubric,doctype_description,head,datum,content_processed,countries,Sentiment,objectivity,kategorie
10,45734790,srf.ch,News - Schweiz,Online medium,Kita-Plätze für Kinder mit Behinderung sind Ma...,2022-03-14,"[Kita, Plätze, Kinder, Behinderung, rar, Das, ...",['Schweiz'],0.278125,0.184375,Lokales
15,46368333,Tages-Anzeiger,Zürich,National daily newspaper,Der Schlieremer Wolf ist nun ein Museumsstück,2022-05-14,"[Rückkehr, Raubtiere, Um, Haar, wäre, erste, W...",['Schweiz'],0.121212,0.090909,Lokales
16,45773390,20 minuten,St. Gallen/Region,Regional daily newspaper,Fall Karm Ahmed: Jetzt gibts Hinweis auf Beton...,2022-03-18,"[OBERNEUNFORN, Im, Dezember, 2007, wurde, Ägyp...",[],0.480000,0.000000,Lokales
38,48093527,blick.ch,People,Online medium,Am 16. Kispi-Ball tanzten Prominente für den g...,2022-11-07,"[Prominente, feierten, zugunsten, Zürcher, Kin...",[],0.630769,0.076923,Lokales
55,47001729,blick.ch,Schweiz,Online medium,Riesige Warteschlangen am Flughafen Zürich,2022-07-17,"[Schon, frühen, Morgenstunden, bildeten, Flugh...",[],0.333333,0.600000,Lokales
...,...,...,...,...,...,...,...,...,...,...,...
153174,45778228,blick.ch,Sport,Online medium,Monsterwelle verschluckt Surfer an der portugi...,2022-03-18,"[Die, Wellen, portugiesischen, Nazaré, berücht...",[],0.000000,0.000000,Lokales
153183,46334857,20 minuten,Bern/Region,Regional daily newspaper,Töffraser fast doppelt so schnell unterwegs,2022-05-11,"[TRIENT, Der, Kantonspolizei, Wallis, Radarkon...",[],0.000000,0.000000,Lokales
153186,45316227,Berner Zeitung,Region,Regional daily newspaper,Keine Wohnung für Autobesitzerinnen und -besitzer,2022-02-02,"[Umstrittener, Parkplatz, Trend, In, Bern, imm...",['Schweiz'],-0.087500,0.125000,Lokales
153191,45756810,blick.ch,Schweiz,Online medium,Stadt Zürich will Sprayereien am Kamin Josefsw...,2022-03-16,"[Seit, vergangenen, Herbst, zierte, Schriftzug...",[],0.423077,0.000000,Lokales
